In [1]:
import numpy as np
import itertools
import trimesh
import k3d
import matplotlib.pyplot as plt
%matplotlib inline

from trimesh.ray.ray_pyembree import RayMeshIntersector
from pypoisson import poisson_reconstruction

from igl import hausdorff
from pytorch3d.loss import chamfer_distance
from torch import FloatTensor

from utils.view import from_pose
# from utils.sampling import get_config

# !conda install -c conda-forge pyembree
# !conda install -c conda-forge igl
# !pip install Cython
# !pip install gym

In [2]:
def illustrate_points(points, plot=None, size=0.1):
    if plot is None:
        plot = k3d.plot(name='points')
    plt_points = k3d.points(positions=points, point_size=size)
    plot += plt_points
    plt_points.shader='3d'
    return plot

def illustrate_mesh(vertices, faces, plot=None):
    if plot is None:
        plot = k3d.plot()
        
    plt_surface = k3d.mesh(vertices, faces,
                           color_map = k3d.colormaps.basic_color_maps.Blues,
                           attribute=vertices[:, 2])

    plot += plt_surface
    return plot


def transform_mesh(mesh, rotation, translation=None, reverse=False):
    # rotation = from_pose([x, y, z], [0, 0, 0]) 
    mesh_ = mesh.copy()
    if reverse:
        if translation is not None:
            mesh_.apply_translation(-translation)
        mesh_.apply_transform(rotation.T)
    else:
        mesh_.apply_transform(rotation)
        if translation is not None:
            mesh_.apply_translation(translation)

    return mesh_


def transform_points(points, rotation, translation=None, reverse=False):
    if translation is None:
        translation = np.zeros((3))
    rotation = rotation[:3, :3]
    
    if reverse:
        return (points - translation).dot(rotation)
    return points.dot(rotation.T) + translation


def generate_sunflower_sphere_points(num_points=100):
    indices = np.arange(0, num_points, dtype=float) + 0.5

    phi = np.arccos(1 - 2 * indices / num_points)
    theta = np.pi * (1 + 5 ** 0.5) * indices

    x, y, z = np.cos(theta) * np.sin(phi), np.sin(theta) * np.sin(phi), np.cos(phi)
    points = np.vstack([x, y, z]).T
    return points, phi, theta


In [3]:
class RaycastingImaging:
    def __init__(self, resolution_image=512, resolution_3d=0.02, projection="ortho"):
        self.resolution_image = resolution_image
        self.resolution_3d = resolution_3d
        self.projection = projection
        # self.fov = [115, 85, 80]

        self.rays_screen_coords, self.rays_origins, self.rays_directions = None, None, None

        
    def prepare(self, scanning_radius):
        # scanning radius is determined from the mesh extent
        self.rays_screen_coords, self.rays_origins, self.rays_directions = generate_rays(
            self.resolution_image, self.resolution_3d, radius=scanning_radius)

    def get_image(self, mesh): #, features):
        if any(value is None for value in [self.rays_screen_coords, self.rays_origins, self.rays_directions]):
            raise DataGenerationException('Raycasting was not prepared')

        # get a point cloud with corresponding indexes
        mesh_face_indexes, ray_indexes, points = ray_cast_mesh(
            mesh, self.rays_origins, self.rays_directions)

        # extract normals
        normals = mesh.face_normals[mesh_face_indexes]

        # compute indexes of faces and vertices in the original mesh
        # hit_surfaces_face_indexes = []
        # for idx, surface in enumerate(features['surfaces']):
        #     surface_face_indexes = np.array(surface['face_indices'])
        #     if np.any(np.isin(surface_face_indexes, mesh_face_indexes, assume_unique=True)):
        #         hit_surfaces_face_indexes.extend(surface_face_indexes)
        # mesh_face_indexes = np.unique(hit_surfaces_face_indexes)
        mesh_face_indexes = np.unique(mesh_face_indexes)
        mesh_vertex_indexes = np.unique(mesh.faces[mesh_face_indexes])
        return ray_indexes, points, normals, mesh_vertex_indexes, mesh_face_indexes

        # assemble mesh fragment into a submesh
        # nbhood = reindex_zerobased(mesh, mesh_vertex_indexes, mesh_face_indexes)
        # return ray_indexes, points, normals, nbhood, mesh_vertex_indexes, mesh_face_indexes

    def points_to_image(self, points, ray_indexes, assign_channels=None):
        xy_to_ij = self.rays_screen_coords[ray_indexes]
        # note that `points` can have many data dimensions
        if None is assign_channels:
            assign_channels = [2]
        data_channels = len(assign_channels)
        image = np.zeros((self.resolution_image, self.resolution_image, data_channels))
        image[xy_to_ij[:, 0], xy_to_ij[:, 1]] = points[:, assign_channels]
        return image.squeeze()

In [4]:
import numpy as np
from trimesh.ray.ray_pyembree import RayMeshIntersector


def generate_rays(image_resolution, resolution_3d, radius=1.0):
    """Creates an array of rays and ray directions used for mesh raycasting.
    :param image_resolution: image resolution in pixels
    :type image_resolution: int or tuple of ints
    :param resolution_3d: pixel 3d resolution in mm/pixel
    :type resolution_3d: float
    :param radius: Z coordinate for the rays origins
    :type resolution_3d: float
    :return:
        rays_screen_coords:     (W * H, 2): screen coordinates for the rays
        rays_origins:           (W * H, 3): world coordinates for rays origins
        ray_directions:         (W * H, 3): unit norm vectors pointing in directions of the rays
    """
    if isinstance(image_resolution, tuple):
        assert len(image_resolution) == 2
    else:
        image_resolution = (image_resolution, image_resolution)
    image_width, image_height = image_resolution

    # generate an array of screen coordinates for the rays
    # (rays are placed at locations [i, j] in the image)
    rays_screen_coords = np.mgrid[0:image_height, 0:image_width].reshape(
        2, image_height * image_width).T    # [h, w, 2]

    # place rays physically in locations determined by screen coordinates,
    # then shift so that camera origin is in the midpoint in the image,
    # and linearly stretch so each pixel takes exactly resolution_3d space
    screen_aspect_ratio = image_width / image_height
    rays_origins = (rays_screen_coords / np.array([[image_height, image_width]]))   # [h, w, 2], in [0, 1]
    factor = image_height / 2 * resolution_3d
    rays_origins[:, 0] = (-2 * rays_origins[:, 0] + 1) * factor  # to [-1, 1] + aspect transform
    rays_origins[:, 1] = (-2 * rays_origins[:, 1] + 1) * factor * screen_aspect_ratio
    rays_origins = np.concatenate([
        rays_origins,
        radius + np.zeros_like(rays_origins[:, [0]])
    ], axis=1)  # [h, w, 3]

    # ray directions are always facing towards Z axis
    ray_directions = np.tile(np.array([0, 0, -1]), (rays_origins.shape[0], 1))

    return rays_screen_coords, rays_origins, ray_directions


def ray_cast_mesh(mesh, rays_origins, ray_directions):
    intersector = RayMeshIntersector(mesh)
    index_triangles, index_ray, point_cloud = intersector.intersects_id(
        ray_origins=rays_origins, ray_directions=ray_directions,
        multiple_hits=False, return_locations=True)
    return index_triangles, index_ray, point_cloud


def make_noise(points, normals, scale=0.0, z_direction=None, **kwargs):
    assert z_direction is not None
    noise = np.random.normal(size=(len(points), 1), scale=scale) * z_direction
    noisy_points = points + noise * z_direction
    return noisy_points

In [5]:
import torch
from utils.matrix_torch import (create_rotation_matrix_x, create_rotation_matrix_y, create_rotation_matrix_z, create_translation_matrix)


class ViewPoint:
    def __init__(self, point, phi, theta):
        self.point = point
        self.phi = phi
        self.theta = theta
        
    def get_transform_matrix(self):
        rotation = torch.mm(create_rotation_matrix_y(-self.phi),
                            create_rotation_matrix_z(-self.theta))
        
        translation = create_translation_matrix(0, 0, 0)
        transform = torch.mm(rotation, translation)
        return transform.t()
        

class Observation:
    def __init__(self, points, normals, vertex_indexes, face_indexes, depth_map, normals_image):
        self.points = points
        self.normals = normals
        self.vertex_indexes = vertex_indexes
        self.face_indexes = face_indexes
        
        if len(depth_map.shape) == 2:
            self.depth_map = np.expand_dims(depth_map, 0)
            self.normals_image = np.expand_dims(normals_image, 0)
        else:
            self.depth_map = depth_map
            self.normals_image = normals_image
    
    
    @property
    def size(self):
        return self.face_indexes.shape[0]
    
    
    def transform(self, transform):
        self.points = transform_points(self.points, transform)
        self.normals = transform_points(self.normals, transform,
                                        translation=None)
        
        
    def __add__(self, other):
        points = np.concatenate([self.points, other.points])
        normals = np.concatenate([self.normals, other.normals])
        
        vertex_indexes = np.unique(np.concatenate([self.vertex_indexes,
                                                   other.vertex_indexes]))
        face_indexes = np.unique(np.concatenate([self.face_indexes,
                                                 other.face_indexes]))
        
        depth_map = np.concatenate([self.depth_map, other.depth_map])
        normals_image = np.concatenate([self.normals_image, other.normals_image])
        
        return Observation(points, normals, vertex_indexes, face_indexes, depth_map, normals_image)
        
        
    def illustrate(self, plot=None, size=0.05):
        return illustrate_points(self.points, plot=plot, size=size)
    
    
class Model:
    def __init__(self, model_path):
        self.mesh = self.load_mesh(model_path)
        self.transform = np.eye(4)
        
        self.raycaster = self.prepare_raycaster()
        
        self.view_points = []
    
    
    def load_mesh(self, mesh_path, shape_fabrication_extent=10.0):
        mesh = trimesh.load_mesh(mesh_path)
        mesh_extent = np.max(mesh.bounding_box.extents)
        mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)
        # TODO compute lengths of curves + quantiles
        mesh = mesh.apply_translation(-mesh.vertices.mean(axis=0))
        return mesh
    
    def prepare_raycaster(self):
        raycaster = RaycastingImaging()
        raycaster.prepare(scanning_radius=np.max(self.mesh.bounding_box.extents) + 1.0)
        return raycaster
    
    def generate_view_points(self, num_points=100):
        sphere_points, phis, thetas = generate_sunflower_sphere_points(num_points)

        dists = self.mesh.vertices - self.mesh.center_mass
        radius = np.abs(dists).max()
        radius *= 1.20

        sphere_points *= radius
        sphere_points += self.mesh.center_mass

        for point, phi, theta in zip(sphere_points, phis, thetas):
            self.view_points.append(ViewPoint(point, phi, theta))
            
            
    def get_point(self, view_point_idx):
        return self.view_points[view_point_idx].point
            

    def illustrate(self):
        plot = illustrate_mesh(self.mesh.vertices, self.mesh.faces)
        plot = illustrate_points([vp.point for vp in self.view_points],
                                 plot, size=0.1)
        return plot


    def rotate_to_view_point(self, view_point):
        self.transform = view_point.get_transform_matrix()
        self.mesh = transform_mesh(self.mesh, self.transform, reverse=True)
        
        
    def rotate_to_origin(self):
        self.mesh = transform_mesh(self.mesh, self.transform, reverse=False)
        self.transform = np.eye(4)

        
    def raycast(self, visibility_eps = 1e-6):
        
        (ray_indexes,
         points,
         normals,
         mesh_vertex_indexes,
         mesh_face_indexes) = self.raycaster.get_image(self.mesh)
        
        noisy_points = make_noise(points, normals, z_direction=np.array([0., 0., -1.]))
        depth_map = self.raycaster.points_to_image(noisy_points, ray_indexes)
        normals_image = self.raycaster.points_to_image(normals, ray_indexes,
                                                       assign_channels=[0, 1, 2])

        observation = Observation(noisy_points,
                                  normals,
                                  mesh_vertex_indexes,
                                  mesh_face_indexes,
                                  depth_map,
                                  normals_image)
        return observation


    def get_observation(self, view_point_idx):
        view_point = self.view_points[view_point_idx]
        self.rotate_to_view_point(view_point)
        
        observation = self.raycast()
        observation.transform(self.transform)
        
        self.rotate_to_origin()
        
        return observation
    
    
    def surface_similarity(self, reconstructed_vertices, reconstructed_faces):
        return hausdorff(self.mesh.vertices,
                         self.mesh.faces,
                         reconstructed_vertices,
                         reconstructed_faces.astype(np.int64))
        

    def observation_similarity(self, observation):
        # TODO: We don't want to deal with GPU tensors right now, so simplify to this
        # chamfer
        # gt = FloatTensor(np.expand_dims(self.mesh.vertices, axis=0))
        # pred = FloatTensor(np.expand_dims(observation.vertices, axis=0))
        # return chamfer_distance(gt, pred)

        # area ratio
        return observation.face_indexes.shape[0] * 1.0 / self.mesh.faces.shape[0]

    
def get_mesh(observation):
    faces, vertices = poisson_reconstruction(
        observation.points, observation.normals, depth=10)
    return vertices, faces


def combine_observations(observations):
    points = np.concatenate([observation.points for observation in observations])
    normals = np.concatenate([observation.normals for observation in observations])

    vertex_indexes = np.unique(np.concatenate([observation.vertex_indexes
                                               for observation in observations]))
    face_indexes = np.unique(np.concatenate([observation.face_indexes
                                             for observation in observations]))

    depth_map = np.concatenate([observation.depth_map for observation in observations])
    normals_image = np.concatenate([observation.normals_image for observation in observations])

    return Observation(points, normals, vertex_indexes, face_indexes, depth_map, normals_image)



In [7]:
from time import sleep

NUM_POINTS = 10


plot = k3d.plot(name='points')
plot.display()

model = Model("./data/Dude.obj")
model.generate_view_points(NUM_POINTS)

observations = []
for view_point_idx in range(NUM_POINTS):
    observation = model.get_observation(view_point_idx)
    
    plot = observation.illustrate(plot, size=0.01)
    plot = illustrate_points([model.get_point(view_point_idx)], size=1.0, plot=plot)
    sleep(2)
    
    observations.append(observation)
    
combined_observation = combine_observations(observations)
reconstructed_vertices, reconstructed_faces = get_mesh(combined_observation)

loss = model.surface_similarity(reconstructed_vertices, reconstructed_faces)
print(loss)

illustrate_mesh(reconstructed_vertices, reconstructed_faces)

Output()

unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!


0.059667802116076925


/opt/conda/lib/python3.7/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

## Environment

In [19]:
import gym
from gym import spaces

MAX_POINS_CNT = 100000
VIEW_POINTS_CNT = 1000


class EnvError(Exception):
    pass


class Environment(gym.Env):
    def __init__(self, number_of_view_points=VIEW_POINTS_CNT):
        super().__init__()

        self.number_of_view_points = number_of_view_points

        self.action_space = spaces.Discrete(number_of_view_points)
        self.observation_space = spaces.Box(
            -np.inf, np.inf, (MAX_POINS_CNT, 3), dtype=np.float32)

        self._similarity_threshold = 1 - 1e-3
        self._reconstruction_depth = 10

        self.model = None
        self.plot = None


    def reset(self):
        """
        Reset the environment for new episode.
        Randomly (or not) generate CAD model for this episode.
        """
        self.model = Model("./data/Dude.obj")
        self.model.generate_view_points(self.number_of_view_points)
        
        self.model.illustrate().display()
        
        init_state = self.action_space.sample()
        observation = self.model.get_observation(init_state)
        return observation


    def step(self, action):
        """
        Get new observation from current position (action), count step reward, decide whether to stop.
        Args:
            action: int
        return: 
            next_state: List[List[List[int, int, int]]]
            reward: float
            done: bool
            info: Tuple
        """
        observation = self.model.get_observation(action)

        reward = self.step_reward(observation)
        done = reward >= self._similarity_threshold

        return observation, reward, done, {}

    
    def render(self, action, observation, plot=None):
        if plot is None:
            plot = self.plot
            
        plot = illustrate_points(
           [self.model.get_point(action)], size=0.5, plot=plot)
        
        plot = observation.illustrate(plot, size=0.01)
        return plot
    
    
    def step_reward(self, observation):
        return self.model.observation_similarity(observation)
    
    
    def final_reward(self, observation, illustrate=False):
        vertices, faces = self._get_mesh(observation)
        reward = self.model.surface_similarity(vertices, faces)
        
        if illustrate:
            illustrate_mesh(vertices, faces).display()
        return reward
        
        
    def _get_mesh(self, observation):
        faces, vertices = poisson_reconstruction(observation.points,
                                                 observation.normals,
                                                 depth=self._reconstruction_depth)
        return vertices, faces


    
class CombiningObservationsWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
        # self.env = environment
        
        # self.action_space = env.action_space
        # self.observation_space = env.observation_space
        
        self._similarity_threshold = 1 - 1e-2

        self.combined_observation = None
        self.plot = None

    
    def reset(self):
        observation = self.env.reset()
        self.combined_observation = observation
        
        self.plot = k3d.plot(name='wrapper')
        self.plot.display()

        
    def step(self, action):
        observation, reward, done, info = self.env.step(action)

        self._combine_observations(observation)

        combined_reward = self.env.step_reward(self.combined_observation)
        done = done or combined_reward >= self._similarity_threshold
        
        new_reward = combined_reward - reward
        print("Step reward: {:.6f}".format(reward),
              "Comb reward: {:.6f}".format(combined_reward),
              "new reward: {:.6f}".format(new_reward),
              "Len observation: ", self.combined_observation.points.shape[0])
        
        return self.combined_observation, new_reward, done, info

    
    def render(self, action, observation):
        self.plot = self.env.render(action, observation, self.plot)
        
    
    def final_reward(self, illustrate=False):
        return self.env.final_reward(self.combined_observation,
                                     illustrate=illustrate)
    

    def _combine_observations(self, observation):
        if self.combined_observation is None:
            raise EnvError("Environment wasn't reset")
        
        self.combined_observation += observation
        
        
class StepPenaltyRewardWrapper(gym.RewardWrapper):
    def __init__(self, environment):
        super().__init__(env)
        # self.env = environment
        
        # self.action_space = env.action_space
        # self.observation_space = env.observation_space
        self._similarity_reward_weight = 1.0

    
#     def reset(self):
#         self.env.reset()
        
        
    def reward(self, reward):
        print("Old reward: ", reward, " New reward: ", -1 + self._similarity_reward_weight * reward)
        reward = -1 + self._similarity_reward_weight * reward
        return reward
    
    
    def render(self, action, observation):
        self.env.render(action, observation)
        
    
    def final_reward(self, illustrate=False):
        return self.env.final_reward(illustrate=illustrate)

    
    
# TODO one more wrapper for GPU (observation, reward -> tensor)

## RandomAgent

In [16]:
class RandomAgent:

    def __init__(self, env):
        self.env = env
        
        self.observation_size = env.observation_space.shape[0]
        self.actions_cnt = env.action_space.n
        
        self._max_iter = 10
        self._gamma = 0.99
        self._final_reward_weight = 1.0
  

    def predict_action(self, state):
        """
        Return action that should be done from input state according to current policy.
        Args:
            state: list of points - results of raycasting
        return: 
            action: int
        """    
        # some cool RL staff
        return self.env.action_space.sample()
    

    def evaluate(self):
        """
        Generate CAD model, reconstruct it and count the reward according   to MSE between original and reconstructed models and number of steps.
        Args:
            environment: Environment
            max_iter: int - max number of iterations to stop (~15)
            gamma: float - discounted factor
            w: float - weight of mse to final episode reward
        return: 
            episode_reward: float
        """    
        
        state = self.env.reset()
        
        episode_reward = 0.0
        states = []
        for t in range(self._max_iter):
            action = self.predict_action(state)
            state, reward, done, info = self.env.step(action)
            print("REWARD: ", reward)
            print()
            states.append(state)
            self.env.render(action, state)
            episode_reward += reward * self._gamma ** t

            if done:
                break
        
        final_reward = self.env.final_reward(illustrate=True)
        print("Hausdorff reward: ", final_reward)
        episode_reward += self._final_reward_weight / final_reward # QUESTION
        return episode_reward, states


In [20]:
env = Environment()
env = CombiningObservationsWrapper(env)
env = StepPenaltyRewardWrapper(env)

agent = RandomAgent(env)

In [18]:
reward, states = agent.evaluate()

unable to load materials from: FinalBaseMesh.mtl
specified material (default)  not loaded!


Output()

Output()

Step reward: 0.275706 Comb reward: 0.451592 new reward: 0.175886 Len observation:  70077
Old reward:  0.17588617686741076  New reward:  -0.8241138231325893
REWARD:  -0.8241138231325893

Step reward: 0.263543 Comb reward: 0.544912 new reward: 0.281369 Len observation:  101652
Old reward:  0.2813688212927757  New reward:  -0.7186311787072244
REWARD:  -0.7186311787072244

Step reward: 0.357394 Comb reward: 0.799256 new reward: 0.441862 Len observation:  137841
Old reward:  0.4418618913283454  New reward:  -0.5581381086716546
REWARD:  -0.5581381086716546

Step reward: 0.333722 Comb reward: 0.840672 new reward: 0.506950 Len observation:  170799
Old reward:  0.5069504068032218  New reward:  -0.4930495931967782
REWARD:  -0.4930495931967782

Step reward: 0.295086 Comb reward: 0.890654 new reward: 0.595568 Len observation:  196483
Old reward:  0.5955680935442986  New reward:  -0.4044319064557014
REWARD:  -0.4044319064557014

Step reward: 0.309988 Comb reward: 0.898994 new reward: 0.589006 Len o

Output()

Hausdorff reward:  0.06039934073458476


## DQN

In [15]:
states[-1].depth_map.shape

(11, 512, 512)